2. Model Evaluation

Classification metrics are important for measuring the performance of your model. Scikit-learn provides several options such as the classification_report and confusion_matrix functions. Another helpful option is the AUC ROC and precision-recall curve. Try to understand what these metrics mean and give arguments why one metric would be more important then others.

For instance, if you have to predict whether a patient has cancer or not, the number of false negatives is probably more important than the number of false positives. This would be different if we were predicting whether a picture contains a cat or a dog – or not: it all depends on the context. Thus, it is important to understand when to use which metric.

For this exercise, you can use your own dataset if that is eligable for supervised classification. Otherwise, you can use the breast cancer dataset which you can find on assemblix2019 (/data/datasets/DS3/). Go through the data science pipeline as you've done before:

Try to understand the dataset globally.

1. Load the data.

2. Exploratory analysis

3. Preprocess data (skewness, normality, etc.)

4. Modeling (cross-validation and training)

5. Evaluation
Create and train several LogisticRegression 
and SVM models with different values for their hyperparameters.

 Make use of the model evaluation techniques that have been described during the plenary part to determine the best model for this dataset.
  Accompany you elaborations with a conclusion, in which you explicitely interpret these evaluation and describe why the different metrics you are using are important or not. Make sure you take the context of this dataset into account.

In [2]:
#load libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix

In [20]:
#1. load data
path = "/Users/Hesam_1/Library/CloudStorage/OneDrive-HanzehogeschoolGroningen/Semester2/MachineLearning/Data/breast-cancer.csv"
df = pd.read_csv(path)
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [21]:
#2. Exploratory analysis
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 32 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       569 non-null    int64  
 1   diagnosis                569 non-null    object 
 2   radius_mean              569 non-null    float64
 3   texture_mean             569 non-null    float64
 4   perimeter_mean           569 non-null    float64
 5   area_mean                569 non-null    float64
 6   smoothness_mean          569 non-null    float64
 7   compactness_mean         569 non-null    float64
 8   concavity_mean           569 non-null    float64
 9   concave points_mean      569 non-null    float64
 10  symmetry_mean            569 non-null    float64
 11  fractal_dimension_mean   569 non-null    float64
 12  radius_se                569 non-null    float64
 13  texture_se               569 non-null    float64
 14  perimeter_se             5

In [22]:
#check for missing values
df.isnull().sum()

id                         0
diagnosis                  0
radius_mean                0
texture_mean               0
perimeter_mean             0
area_mean                  0
smoothness_mean            0
compactness_mean           0
concavity_mean             0
concave points_mean        0
symmetry_mean              0
fractal_dimension_mean     0
radius_se                  0
texture_se                 0
perimeter_se               0
area_se                    0
smoothness_se              0
compactness_se             0
concavity_se               0
concave points_se          0
symmetry_se                0
fractal_dimension_se       0
radius_worst               0
texture_worst              0
perimeter_worst            0
area_worst                 0
smoothness_worst           0
compactness_worst          0
concavity_worst            0
concave points_worst       0
symmetry_worst             0
fractal_dimension_worst    0
dtype: int64

From this exploratory data analysis, it seems the data is clean as there are no missing values. 
However, the 'diagnosis' feature is an object which will need to be encoded to numerical values since machine learning models work best with numerical data.
 The 'id' column is also unnecessary for the modeling process, so we can drop it.

Let's proceed to preprocess the data. Afterwards, we will train Logistic Regression and SVM models with various hyperparameters and use cross-validation to evaluate them.

In [23]:
df['diagnosis'].value_counts()

B    357
M    212
Name: diagnosis, dtype: int64

In [24]:
# 3. Preprocessing
# Encoding categorical data

labelencoder = LabelEncoder()
df['diagnosis'] = labelencoder.fit_transform(df['diagnosis'])

In [25]:
df['diagnosis'].value_counts()

0    357
1    212
Name: diagnosis, dtype: int64

In [26]:
# Dropping the 'id' column
df = df.drop('id', axis=1)


# Splitting data into features and target variable
X = df.drop('diagnosis', axis=1)
y = df['diagnosis']

In [28]:
# Scaling the data
sc = StandardScaler()
df[X.columns] = sc.fit_transform(df[X.columns])

In [29]:


# Now split your data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)





In [31]:


# 4.  Modeling
# Grid of hyperparameters for Logistic Regression and SVM
param_grid_lr = {'C': [0.1, 1, 10, 100], 'penalty': ['l1', 'l2']}
param_grid_svc = {'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01, 0.001], 'kernel': ['rbf', 'poly', 'sigmoid']}

# # Hyperparameters for models
# param_grid_lr = {'C': [0.001, 0.01, 0.1, 1, 10, 100], 'penalty': ['l2']}
# param_grid_svc = {'C': [0.001, 0.01, 0.1, 1, 10, 100], 'gamma': [0.001, 0.01, 0.1, 1, 10, 100]}

# Grid search with cross-validation
grid_lr = GridSearchCV(LogisticRegression(), param_grid_lr, refit=True)
grid_svc = GridSearchCV(SVC(), param_grid_svc, refit=True)

# Training the models
grid_lr.fit(X_train, y_train)
grid_svc.fit(X_train, y_train)



/Users/Hesam_1/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/Hesam_1/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logisti

In [ ]:
#  Evaluation
# Evaluating Logistic Regression
y_pred_lr = grid_lr.predict(X_test)
print("Best parameters for Logistic Regression: ", grid_lr.best_params_)
print(classification_report(y_test, y_pred_lr))
print(confusion_matrix(y_test, y_pred_lr))

# Evaluating SVM
y_pred_svc = grid_svc.predict(X_test)
print("Best parameters for SVM: ", grid_svc.best_params_)
print(classification_report(y_test, y_pred_svc))
print(confusion_matrix(y_test, y_pred_svc))